# Title
[]()

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
from silvhua import *

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Initiate

In [4]:
result_dict = dict()


# Iteration 1

In [5]:
# from openai import OpenAI
import boto3
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

def create_system_message(
        business_name, 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        ):
    instructions_filename = f'{business_name}.md'
    try:
        instructions = load_txt(instructions_filename, prompts_filepath)
        # examples = load_txt(f'{business_name}.txt', examples_filepath)
        # document = load_txt(f'{business_name}_doc.md', doc_filepath)
    except:
        s3 = boto3.client('s3')
        instructions = s3.get_object(Bucket='ownitfit-silvhua', Key=instructions_filename)
    return instructions

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

iteration = 1
result_dict[iteration] = create_system_message(
        'CoachMcloone', 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        )

In [17]:
# load the s3 object
print(result_dict[iteration]['Body'].read().decode('utf-8'))

In [19]:
result_dict[iteration]['Body'].read()

b''

In [ ]:
iteration = 1.01
result_dict[iteration] = create_system_message(
        'Coach', 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        )
result_dict[iteration]

In [ ]:
iteration = 1.02
result_dict[iteration] = create_system_message(
        'CoachMcloone', 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        )
result_dict[iteration]

## 1.1

In [ ]:
# from openai import OpenAI
import boto3
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

def create_system_message(
        business_name, 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        ):
    instructions_filename = f'{business_name}.md'
    try:
        instructions = load_txt(instructions_filename, prompts_filepath)
        # examples = load_txt(f'{business_name}.txt', examples_filepath)
        # document = load_txt(f'{business_name}_doc.md', doc_filepath)
    except:
        s3 = boto3.client('s3')
        instructions = s3.get_object(
            Bucket='ownitfit-silvhua', Key=instructions_filename
            )['Body'].read().decode('utf-8')
    return instructions

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

iteration = 1.1
result_dict[iteration] = create_system_message(
        'CoachMcloone', 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        )
result_dict[iteration] 

## 1.11

In [ ]:
# from openai import OpenAI
import boto3
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

def create_system_message(
        business_name, 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        ):
    instructions_filename = f'{business_name}.md'
    examples_filename = f'{business_name}.txt'
    document_filename = f'{business_name}_doc.md'
    try:
        instructions = load_txt(instructions_filename, prompts_filepath)
        examples = load_txt(examples_filename, examples_filepath)
        document = load_txt(document_filename, doc_filepath)
    except:
        s3 = boto3.client('s3')
        instructions = s3.get_object(
            Bucket='ownitfit-silvhua', Key=instructions_filename
            )['Body'].read().decode('utf-8')
        examples = s3.get_object(
            Bucket='ownitfit-silvhua', Key=examples_filename
            )['Body'].read().decode('utf-8')
        document = s3.get_object(
            Bucket='ownitfit-silvhua', Key=document_filename
            )['Body'].read().decode('utf-8')

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

iteration = 1.11
result_dict[iteration] = create_system_message(
        'CoachMcloone', 
        prompts_filepath='app/private/prompts',
        examples_filepath='app/private/data/chat_examples', doc_filepath='app/private/data/rag_docs'
        )
result_dict[iteration] 

In [ ]:
print(result_dict[iteration] )

# filter out emplated emails

In [36]:
event = {
  "body": {
    "type": "OutboundMessage",
    "locationId": "fsdVH26v30hoBBQOBttG",
    "attachments": [],
    "body": "This is a sample email. click here to unsubscribe. Link.",
    "contactId": "6kIAuqeiBUxJ3nWTwNVE",
    "contentType": "text/plain",
    "conversationId": "fcanlLgpbQgQhderivVs",
    "dateAdded": "2024-01-13T05:15:00.000Z",
    "direction": "outbound",
    "messageType": "Email",
    "userId": "80pzXdC3MCqKRpzrteuj"
  },
  "direct_local_invoke": 1
}
payload = event["body"]
message = payload.get("body")
# see if message contains the substring "unsubscribe"
if "click here to unsubscribe" in message.lower():
    print("yes")

yes


# *End of Page*